<a href="https://colab.research.google.com/github/joni901/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


!pip install PyMuPDF transformers spacy nltk simpletransformers language-tool-python rake_nltk  sentence-transformers torch
!pip install -U sentence-transformers pinecone-client
!python -m spacy download pt_core_news_sm
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
import fitz  # PyMuPDF
import re
import torch
import spacy
import unicodedata
from transformers import T5ForConditionalGeneration, T5Tokenizer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import language_tool_python
from rake_nltk import Rake
import nltk
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pinecone
from sklearn.preprocessing import normalize
import tensorflow as tf
import tensorflow_hub as hub

# Inicializar Spacy e NLTK

# Inicializar LanguageTool
tool = language_tool_python.LanguageTool('pt-BR')

# Função para corrigir texto usando LanguageTool
def corrigir_texto(texto):
    matches = tool.check(texto)
    texto_corrigido = language_tool_python.utils.correct(texto, matches)
    return texto_corrigido

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

pdf_path = "/content/Relatório Final de Projetos Propostos 2021 com errata (1).pdf"
text = extract_text_from_pdf(pdf_path)



# Função para limpar o texto
def limpar_texto(texto):
    texto_limpo = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    texto_limpo = re.sub(r'\d+/\d+\s*-\s*CADERNOS DE ELEI.*?2013', '', texto_limpo)
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
    texto_limpo = re.sub(r'\n', ' ', texto_limpo)

    return texto_limpo.strip()

# Carregar o modelo de transformadores de sentenças

def chunk_text(text, model):
    # Dividir o texto em sentenças individuais
    single_sentences = re.split(r'(?<=\.|\?)\s', text)
    ss = [limpar_texto(i) for i in single_sentences]

    # Preparar os dados para processamento
    sentences = [{"sentence": x, "index": y} for y, x in enumerate(ss)]

    # Função para combinar sentenças adjacentes
    def combine_sentences(sentences, max_chunk_size=1):
        combined_sentences = []
        for i in range(len(sentences)):
            combined_sentence = sentences[i]["sentence"]
            for j in range(1, max_chunk_size + 1):
                if i + j < len(sentences):
                    combined_sentence += " " + sentences[i + j]["sentence"]
            combined_sentences.append(combined_sentence)
        return combined_sentences

    combined_sentences = combine_sentences(sentences)

    # Obter embeddings para as sentenças combinadas
    embeddings = model(combined_sentences).numpy()

    for i, x in enumerate(sentences):
        sentences[i]["combined_sentences_embedding"] = embeddings[i]

    # Função para calcular distâncias cosseno entre embeddings
    def calculate_cosine_distances(sentences):
        distances = []
        for i in range(len(sentences) - 1):
            embedding_current = sentences[i]["combined_sentences_embedding"]
            embedding_next = sentences[i + 1]["combined_sentences_embedding"]

            similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
            distance = 1 - similarity
            distances.append(distance)
            sentences[i]["distance_next"] = distance
        return distances, sentences

    distances, sentences = calculate_cosine_distances(sentences)

    breakpoint_percentile_threshold = 90

    breakpoint_percentile = np.percentile(distances, breakpoint_percentile_threshold)

    # Identificar os índices dos pontos de quebra
    indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_percentile]

    # Marcar os chunks no gráfico
    start_index = 0
    chunks = []
    for index in indices_above_thresh:
        end_index = index
        group = sentences[start_index:end_index + 1]
        combined_text = " ".join([x["sentence"] for x in group])
        chunks.append(combined_text.strip())
        start_index = end_index + 1

    if start_index < len(sentences):
        group = sentences[start_index:]
        combined_text = " ".join([x["sentence"] for x in group])
        chunks.append(combined_text.strip())

    return chunks

model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


chunks = chunk_text(text, model)
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}: {chunk}\n")


# Função para extrair palavras-chave usando Rake
def extrair_palavras_chave_rake(texto, top_n=5):
    rake = Rake(language='portuguese')
    rake.extract_keywords_from_text(texto)
    palavras_chave = rake.get_ranked_phrases()[:top_n]
    return palavras_chave

# Carregar modelo T5 e tokenizer para geração de títulos
model_name = "pierreguillou/t5-base-qa-squad-v1.1-portuguese"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Função para gerar título a partir do texto
def generate_title(text, max_length=20):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=max_length, num_beams=4, early_stopping=True)
    title = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return title


# Dividindo o texto em partes gerenciáveis



generated_titles = {idx: generate_title(content) for idx, content in enumerate(chunks)}


document_titles = list(generated_titles.values())


INFO:language_tool_python.download_lt:Unzipping /tmp/tmprxzs4zd3.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


Chunk 1: Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao Paulo (Incluindo os suplentes que chegaram a ser empossados em

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [3]:
print(chunks)


['Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao Paulo (Incluindo os suplentes que chegaram a ser empossados em algum 

In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def select_best_chunks(chunks, question, top_n=1):
    vectorizer = TfidfVectorizer()
    document_tfidf = vectorizer.fit_transform(chunks)
    question_tfidf = vectorizer.transform([question])
    similarities = cosine_similarity(question_tfidf, document_tfidf).flatten()
    sorted_indices = similarities.argsort()[::-1]
    best_chunks = [chunks[i] for i in sorted_indices[:top_n]]
    return best_chunks

def process_question_tfidf(question, vectorizer):
    question=question.lower()
    question = corrigir_texto(question)
    return vectorizer.transform([question])

def search_index_tfidf(question, vectorizer, document_tfidf, document_titles, document_value):
    question_tfidf = process_question_tfidf(question, vectorizer)
    similarities = cosine_similarity(question_tfidf, document_tfidf).flatten()
    sorted_indices = similarities.argsort()[::-1]
    return {document_titles[idx]: document_value[idx] for idx in sorted_indices}

# Preparação dos dados
vectorizer = TfidfVectorizer()
document_tfidf = vectorizer.fit_transform(chunks)

question = input("Digite sua pergunta: ")
results = search_index_tfidf(question, vectorizer, document_tfidf, document_titles, chunks)
selected_chunks = select_best_chunks(list(results.values()), question)

print("Melhor trecho selecionado:", selected_chunks)

Digite sua pergunta: qual objetivo do pdf
Melhor trecho selecionado: ['Avenida da Liberdade, 532 Bloco E 4o andar Sala Observatorio Social Sao Paulo - SP CEP 01502-001 Fones: (11) 3272-4288 ou (11) 96460-0435 E-mail: saopaulo@osbrasil.org.br RELATORIO DA ANALISE DE PROJETOS PROPOSTOS NO ANO DE 20211 INTRODUCAO O grupo de monitoramento do legislativo do Observatorio Social do Brasil de Sao Paulo realizou o levantamento e a analise da atividade de projetos propostos do ano de 2021, de forma igual ao que vinha realizando desde o ano de 2017. Como metodologia para a realizacao da atividade em 2021, foram distribuidas entre os voluntarios planilhas individuais dos vereadores, para o levantamento dos projetos propostos (Fonte: https://splegisconsulta.saopaulo.sp.leg.br/) e sua classificacao em categorias, pre-estabelecidas pela equipe. Tendo em vista que as categorias trazem um certo grau de subjetividade, apos os dados serem levantados, as planilhas foram enviadas para os vereadores de Sao 